<a href="https://colab.research.google.com/github/kmachida12345/openai-playground/blob/main/search_faq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# パッケージのインスコ

In [ ]:
!pip install openai

# 環境変数の読み込み

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

# Embeddings生成

csvはtitleとcontentsを持っている想定。
Embeddingsを生成してrowに追加した新しいデータを作って保存する。

In [ ]:
import pandas as pd
import openai
import time

df = pd.read_csv("faq_list.csv", on_bad_lines='skip')

dfList = []
for index, row in df.iterrows():
    print(index) #To keep track of which row we are on

    entry = {}
    entry['question'] = row['question']
    entry['answer'] = row['answer']

    entry['embedding'] = openai.Embedding.create(input = row['question'], model="text-embedding-ada-002")['data'][0]['embedding']
    
    dfList.append(entry)

    time.sleep(1) #To avoid rate limiting

df = pd.DataFrame(dfList)
df.to_csv("faq_embeddings.csv", index=False)

# 生成したEmbeddingsを検索

Embeddingsの列だけ抜き出して検索をかけ、ヒットしたインデックスを元のデータと突き合わせて表示する

In [ ]:
import pandas as pd
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

df = pd.read_csv("faq_embeddings.csv", on_bad_lines='skip')

query = input("word or sentence:")

# Convert query to embedding
query_embedding = openai.Embedding.create(input=query, model="text-embedding-ada-002")['data'][0]['embedding']

# Create a new column in the dataframe that contains the embeddings as numpy arrays
embeddings = df['embedding'].apply(eval).apply(np.array)

# Calculate the distances between the query embedding and the summary embeddings using the cosine distance metric
distances = distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")

# Get the indices of the nearest neighbors
indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

print("found:")
print(df.loc[indices_of_nearest_neighbors[:5]]['title'])

print("contents of each:")
print(df.loc[indices_of_nearest_neighbors[:5]]['contents'])

